## 타이타닉 생존자 예측
- 데이터 읽기
- 데이터 전처리
- 학습용 데이터 만들기
- 모델생성
- 모델검증
- 모델향상

In [56]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
raw_df = pd.read_csv('titanic_train.csv')

In [10]:
cleansing_feature = ['PassengerId','Name']
cl_df = raw_df.drop(cleansing_feature,axis=1)

In [11]:
cl_df.Embarked.fillna('N',inplace=True)

In [16]:
cl_df.Cabin.fillna('N',inplace=True)

In [18]:
cl_df.Age.fillna(round(cl_df.Age.mean(),1),inplace=True)

In [29]:
cl_df['Cabin_tmp'] = cl_df.Cabin.str[:1]

In [38]:
gender_encoder = LabelEncoder()
gender_encoder.fit(cl_df.Sex)
gender = gender_encoder.transform(cl_df.Sex)
Cabin_encoder = LabelEncoder()
Cabin_encoder.fit(cl_df.Cabin_tmp)
Cabin_class = Cabin_encoder.transform(cl_df.Cabin_tmp)
Embarked_encoder = LabelEncoder()
Embarked_encoder.fit(cl_df.Embarked)
Embarked_class = Embarked_encoder.transform(cl_df.Embarked)

In [35]:
cl_df['gender'] = gender

In [39]:
cl_df['Cabin_class'] = Cabin_class
cl_df['Embarked_class'] = Embarked_class

In [42]:
feature_cols = ['Pclass','Age','SibSp','Parch','gender',
                'Cabin_class','Embarked_class']
y_col = ['Survived']
X = cl_df[feature_cols]
y = cl_df[y_col]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3)

In [47]:
tree_model = DecisionTreeClassifier()

In [48]:
tree_model.fit(X_train,y_train)

DecisionTreeClassifier()

In [49]:
tree_pred = tree_model.predict(X_test)

In [51]:
accuracy_score(y_test,tree_pred)

0.7798507462686567

In [53]:
svc_model = SVC()
svc_model.fit(X_train,y_train)
svc_pred = svc_model.predict(X_test)
accuracy_score(y_test,svc_pred)

C:\anaconda3\envs\keras\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.7126865671641791

In [55]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
rf_pred = rf_model.predict(X_test)
accuracy_score(y_test,rf_pred)

<ipython-input-55-be20a40645d8>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train,y_train)


0.7947761194029851

In [57]:
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
lr_pred = lr_model.predict(X_test)
accuracy_score(y_test,lr_pred)

C:\anaconda3\envs\keras\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\anaconda3\envs\keras\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7835820895522388

In [69]:
lr_model.predict_proba(np.array(X_test.iloc[0,:]).reshape(1,-1))
lr_model.predict_proba(X_test.iloc[0:2,:])

array([[0.05885985, 0.94114015],
       [0.90659833, 0.09340167]])

In [68]:
a1 = RandomForestClassifier()
a2 = LogisticRegression()
a3 = KNeighborsClassifier()

## Hard voting
- 각 분류기마다 모델생성
- 분류기부터 예측결과 취합
- 분류기가 예측한 투표 결과를 다수결로 하여 예측결과 산정
- accuracy_score(y_test,예측결과)

In [72]:
a1.fit(X_train.values,y_train.values)
a2.fit(X_train.values,y_train.values)
a3.fit(X_train.values,y_train.values)
a1_pr = a1.predict(X_test.values)
a2_pr = a2.predict(X_test.values)
a3_pr = a3.predict(X_test.values)

<ipython-input-72-2b02a9a2e371>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  a1.fit(X_train.values,y_train.values)
C:\anaconda3\envs\keras\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\anaconda3\envs\keras\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<ipyth

In [77]:
hv = (a1_pr+a2_pr+a3_pr)
hv_result = np.where(hv >= 2,1,0)

In [78]:
accuracy_score(y_test,hv_result)

0.8246268656716418

## 소프트 보팅
- 각 분류기마다 모델생성
- 분류기부터 예측결과 취합(확률값)
- 분류기가 예측한 투표 결과를 확률의 기대값으로 예측결과 산정
- accuracy_score(y_test,예측결과)

In [80]:
a1.fit(X_train.values,y_train.values)
a2.fit(X_train.values,y_train.values)
a3.fit(X_train.values,y_train.values)
a1_pr = a1.predict_proba(X_test.values)
a2_pr = a2.predict_proba(X_test.values)
a3_pr = a3.predict_proba(X_test.values)

<ipython-input-80-907aced81190>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  a1.fit(X_train.values,y_train.values)
C:\anaconda3\envs\keras\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\anaconda3\envs\keras\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<ipyth

In [90]:
sv = np.where((a1_pr+a2_pr+a3_pr)[:,0] 
         - (a1_pr+a2_pr+a3_pr)[:,1] >0,0,1)

In [91]:
accuracy_score(y_test,sv)

0.832089552238806